# Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building the **credit score** of a potential customer. The **credit score** is used to evaluate the ability of a potential borrower to repay their loan.


the purpose of this project is to explore the connection between a customer's marital status and number of children to chances of defaulting on a loan.

## Open the data file and have a look at the general information. 


Loading all the libraries

In [130]:

import pandas as pd
import numpy as np
import nltk
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from pymystem3 import mystem
# m = mystem() 
wordnet_lemma = WordNetLemmatizer()
from collections import Counter




load the data

In [131]:

try:
    path='C:\\Users\\aviv\\Downloads\\credit_scoring_eng.csv'
    df = pd.read_csv(path)
    display(df.head())
except:
    path='/datasets/credit_scoring_eng.csv'
    df = pd.read_csv(path)
    display(df.head())



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


##  Data exploration

**Description of the data**
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan

exploring our data:

Let's see how many rows and columns our dataset has

In [132]:

df.shape


(21525, 12)

let's print the first 10 rows

In [133]:

print(df.head(10))



   children  days_employed  dob_years            education  education_id  \
0         1   -8437.673028         42    bachelor's degree             0   
1         1   -4024.803754         36  secondary education             1   
2         0   -5623.422610         33  Secondary Education             1   
3         3   -4124.747207         32  secondary education             1   
4         0  340266.072047         53  secondary education             1   
5         0    -926.185831         27    bachelor's degree             0   
6         0   -2879.202052         43    bachelor's degree             0   
7         0    -152.779569         50  SECONDARY EDUCATION             1   
8         2   -6929.865299         35    BACHELOR'S DEGREE             0   
9         0   -2188.756445         41  secondary education             1   

       family_status  family_status_id gender income_type  debt  total_income  \
0            married                 0      F    employee     0     40620.102   
1

i see some negative 'days_employed' values which don't make sense also too large. some 'education' values that are all caps, all lower, or mixed

In [134]:
print(df.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None



i see there are missing 'days_employed' values, missing 'total income' values, both same number of missing values 

In [135]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Let's look at the filtered table with missing values in the the first column with missing data

In [136]:


print(df.loc[df['total_income'].isnull(),'total_income'])
print(df.loc[df['days_employed'].isnull(),'days_employed'])
#print(df.loc[df['children'].isnull()].count())





12      NaN
26      NaN
29      NaN
41      NaN
55      NaN
         ..
21489   NaN
21495   NaN
21497   NaN
21502   NaN
21510   NaN
Name: total_income, Length: 2174, dtype: float64
12      NaN
26      NaN
29      NaN
41      NaN
55      NaN
         ..
21489   NaN
21495   NaN
21497   NaN
21502   NaN
21510   NaN
Name: days_employed, Length: 2174, dtype: float64



it seems the rows where 'days employed' and 'total income' are missing match up exactly. Let's check that by boolean indexing with two conditions:

Let's apply multiple conditions for filtering data and look at the number of rows in the filtered table.

In [137]:



print(df.loc[df['days_employed'].isnull() & df['total_income'].isnull(),'days_employed'])

print('percentage of missing values: {rate:%}'.format(rate = 2174 / df.shape[0]))





12      NaN
26      NaN
29      NaN
41      NaN
55      NaN
         ..
21489   NaN
21495   NaN
21497   NaN
21502   NaN
21510   NaN
Name: days_employed, Length: 2174, dtype: float64
percentage of missing values: 10.099884%


**Intermediate conclusion**

the number of rows and missing values are the same. i conclude the missing values are symmetrical.

the percentage of the missing values compared to the whole dataset is about 10%, which is considerably large. i'll definitely fill the missing data


to determine if the missing data could be due to some characteristic i'll print some rows where the values are missing to see whether there's any dependence on the values of other columns.

let's investigate clients who do not have data on identified characteristic and the column with the missing values


In [138]:

print(df.loc[df['total_income'].isnull()].tail())


       children  days_employed  dob_years            education  education_id  \
21489         2            NaN         47  Secondary Education             1   
21495         1            NaN         50  secondary education             1   
21497         0            NaN         48    BACHELOR'S DEGREE             0   
21502         1            NaN         42  secondary education             1   
21510         2            NaN         28  secondary education             1   

           family_status  family_status_id gender income_type  debt  \
21489            married                 0      M    business     0   
21495  civil partnership                 1      F    employee     0   
21497            married                 0      F    business     0   
21502            married                 0      F    employee     0   
21510            married                 0      F    employee     0   

       total_income                 purpose  
21489           NaN       purchase of a car  


In [139]:
# Checking distribution for total income column where the values are missing
df[df['total_income'].isnull()]['income_type'].value_counts(normalize=True)

employee         0.508280
business         0.233671
retiree          0.189972
civil servant    0.067617
entrepreneur     0.000460
Name: income_type, dtype: float64

i checked for correlation between income type and the missing values. i didn't find any.

**Possible reasons for missing values in data**


i can't say exactly why this is, but looks like a problem with the gathering of the data. to me it looks pretty random.
now to check the distribution in the whole dataset


In [140]:

df['income_type'].value_counts(normalize=True)



employee                       0.516562
business                       0.236237
retiree                        0.179141
civil servant                  0.067782
unemployed                     0.000093
entrepreneur                   0.000093
student                        0.000046
paternity / maternity leave    0.000046
Name: income_type, dtype: float64

**Intermediate conclusion**

the distribution of income types are similar between the filtered table and the whole dataset. that means there probably isn't a connection between income types and the reason for missing values.

Checking for other reasons and patterns that could lead to missing values

In [141]:


display(df[df['total_income'].isnull()]['children'].value_counts())
display(df['children'].value_counts())

display(df[df['total_income'].isnull()]['dob_years'].value_counts())
display(df['dob_years'].value_counts())

display(df['education'].value_counts())

display(df[df['total_income'].isnull()]['family_status'].value_counts())
display(df['family_status'].value_counts())

display(df[df['total_income'].isnull()]['gender'].value_counts())
display(df['gender'].value_counts())


display(df[df['total_income'].isnull()]['debt'].value_counts())
display(df['debt'].value_counts())

display(df[df['total_income'].isnull()]['purpose'].value_counts())
display(df['purpose'].value_counts())






 0     1439
 1      475
 2      204
 3       36
 20       9
 4        7
-1        3
 5        1
Name: children, dtype: int64

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

34    69
40    66
42    65
31    65
35    64
36    63
47    59
41    59
30    58
28    57
58    56
57    56
54    55
56    54
38    54
52    53
37    53
33    51
39    51
50    51
43    50
45    50
49    50
51    50
29    50
46    48
55    48
48    46
44    44
53    44
60    39
62    38
61    38
32    37
64    37
23    36
27    36
26    35
59    34
63    29
25    23
24    21
65    20
66    20
21    18
22    17
67    16
0     10
68     9
71     5
69     5
20     5
70     3
72     2
19     1
73     1
Name: dob_years, dtype: int64

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64

...

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

0    2004
1     170
Name: debt, dtype: int64

0    19784
1     1741
Name: debt, dtype: int64

having a wedding                            92
to have a wedding                           81
wedding ceremony                            76
construction of own property                75
housing transactions                        74
buy real estate                             72
purchase of the house for my family         71
transactions with my real estate            71
transactions with commercial real estate    70
housing renovation                          70
buy commercial real estate                  67
buying property for renting out             65
property                                    62
real estate transactions                    61
buy residential real estate                 61
housing                                     60
building a property                         59
cars                                        57
going to university                         56
to become educated                          55
second-hand car purchase                    54
buying my own

wedding ceremony                            797
having a wedding                            777
to have a wedding                           774
real estate transactions                    676
buy commercial real estate                  664
buying property for renting out             653
housing transactions                        653
transactions with commercial real estate    651
purchase of the house                       647
housing                                     647
purchase of the house for my family         641
construction of own property                635
property                                    634
transactions with my real estate            630
building a real estate                      626
buy real estate                             624
building a property                         620
purchase of my own house                    620
housing renovation                          612
buy residential real estate                 607
buying my own car                       

**Intermediate conclusion**

based on the distributions all being similar across all columns and the rows with missing values, i conclude that the values are missing at random.
i also found 'children' values of 20, and nothing above 5 except that, which makes me think that it's a wrong value; if people really do have 20 children, than people also have a little less like 17 or at least 7. also -1 children.

 in the 'dob_years' column i found values of 0. in addition i've found various duplicates in the 'education' column, caused by different capitalizations.
 
furthermore, in the purpose column there are several purposes which could be united into one category (cars, education, etc).
i've also noticed the values in the days employed column are too large, perhaps they are work hours?

**Conclusions**

the distributions for the values at the missing valued rows and the distributions across the whole dataset are similar. so no pattern there.


the missing values are at the 'total_income' and 'days_employed' columns. these are quantitative variables, therefore we could fill the missing values with the mean or the median, depending on how many outliers there are.


**transformation plan** :
missing values, quantitative: 
filling in mean or median depending on existence of outliers.

missing values, categorical:
if they'll be missing at random, filling them with default values.

different registers - using the .lower() method to make the values consistently lowercase

duplicates - check for them with duplicates().count(), drop with drop_duplicates if necessary
incorrect values - large values of days employed could be divided to give the actual days, negative values will be reversed.


## Data transformation



Let's see all values in education column to check if and what spellings will need to be fixed

In [142]:


df['education'].value_counts()


secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64

Fix the registers if required

In [143]:

df['education'] = df['education'].str.lower()

Checking all the values in the column to make sure we fixed them

In [144]:


df['education'].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

Let's see the distribution of values in the `children` column

In [145]:


print(df['children'].value_counts())
print(df['children'].value_counts(normalize=True))



 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
 0     0.657329
 1     0.223833
 2     0.095470
 3     0.015331
 20    0.003531
-1     0.002184
 4     0.001905
 5     0.000418
Name: children, dtype: float64


 there's the -1 which is .2% of the data, and the 20 which makes up .3% of the data.the 20 may have occured due to the closeness of 2 and 0 on the numpad. on the other hand why hasn't this happened with 1 as well? I can't be sure of the reason, and for the -1 i'd guess it's supposed to be 1 and not -1, i'll replace them with 1. in the case of 20 i'll replace with the mean value since we do not have outliers so we can use the average.

In [146]:

mean_children = df.loc[(df['children'] != 20) & (df['children'] != -1), ['children']].mean()
#df.loc[(df['children'] != 20) & (df['children'] != -1), ['children']].median()

df.loc[df['children'] == -1, ['children']] = 1
df.loc[df['children'] == 20, ['children']] = mean_children
print(df['children'].value_counts())




0.0    14149
1.0     4865
2.0     2055
3.0      330
4.0       41
5.0        9
Name: children, dtype: int64



we already saw the negative values, but there can be other issues like really large values (millions of days)

checking for problematic data in `days_employed`, if it exisst, and calculating the percentage

In [147]:


display(df['days_employed'].value_counts())


print(df[df['days_employed'] < 0]['days_employed'].value_counts().shape[0] / df.shape[0])




-8437.673028      1
-3507.818775      1
 354500.415854    1
-769.717438       1
-3963.590317      1
                 ..
-1099.957609      1
-209.984794       1
 398099.392433    1
-1271.038880      1
-1984.507589      1
Name: days_employed, Length: 19351, dtype: int64

0.7389547038327526


73% of the values are negative! we'll turn them positive.


the amount of negative values looks high enough that there's some technical issue that turns the values negative. we'll just turn them positive. can't think of an obvious reason.

In [148]:
# Address the problematic values, if they exist
df['days_employed'] = df.loc[df['days_employed'] < 0]['days_employed'] * (-1)

#df['days_employed'].value_counts()
df.days_employed.describe()


count    15906.000000
mean      2353.015932
std       2304.243851
min         24.141633
25%        756.371964
50%       1630.019381
75%       3157.480084
max      18388.949901
Name: days_employed, dtype: float64

the range of the values definitely seems weird. people have been working for years?

In [149]:
# Check the result - make sure it's fixed
df[df['days_employed'] < 0]['days_employed'].value_counts()

Series([], Name: days_employed, dtype: int64)

In [150]:
df.loc[df['days_employed'] > (365*20)]['dob_years'].value_counts()


49    48
44    44
50    42
52    38
42    37
53    37
51    36
48    34
55    34
47    34
46    33
45    30
43    28
57    27
56    26
59    25
54    25
58    21
41    19
39    18
61    17
40    15
38    15
60    15
63    11
64     8
36     7
62     6
65     6
66     5
0      3
37     3
35     2
67     2
72     2
71     1
69     1
70     1
68     1
Name: dob_years, dtype: int64

values don't seem too large, every person who's worked more than 40 years is at least 50 years old and that makes sense.


however i've noticed a zero value in the 'dob_years' column.

issues in age column:

In [151]:
# Check the `dob_years` for suspicious values and count the percentage
df['dob_years'].value_counts() / df.shape[0]
# there are 101 zero values. otherwise the data seems fine



35    0.028664
40    0.028293
41    0.028200
34    0.028014
38    0.027782
42    0.027735
33    0.026992
39    0.026620
31    0.026016
36    0.025784
44    0.025412
29    0.025319
30    0.025087
48    0.024994
37    0.024948
50    0.023879
43    0.023833
32    0.023693
49    0.023600
28    0.023368
45    0.023089
27    0.022904
56    0.022625
52    0.022485
47    0.022300
54    0.022253
46    0.022067
58    0.021417
57    0.021370
53    0.021324
51    0.020813
59    0.020627
55    0.020581
26    0.018955
60    0.017515
25    0.016585
61    0.016492
62    0.016353
63    0.012497
64    0.012311
24    0.012265
23    0.011800
65    0.009013
22    0.008502
66    0.008502
67    0.007758
21    0.005157
0     0.004692
68    0.004599
69    0.003949
70    0.003020
71    0.002695
20    0.002369
72    0.001533
19    0.000650
73    0.000372
74    0.000279
75    0.000046
Name: dob_years, dtype: float64



i'll replace the zero values with the median of the ages, since from what i saw there are outliers (values which are rarer)

issues in the `dob_years` column, if they exist

In [152]:

df.loc[df['dob_years'] != 0, 'dob_years'].median()

#the median is 43

df.loc[df['dob_years'] == 0, 'dob_years'] = 43


In [153]:
# Check the result - make sure it's fixed
df['dob_years'].value_counts()

35    617
43    614
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

checking family status column for problems:

In [154]:
# Let's see the values for the column

df['family_status'].value_counts()

#no problem there.



married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

problems in gender column:

Let's see the values in the column


In [155]:

df['gender'].value_counts()


F      14236
M       7288
XNA        1
Name: gender, dtype: int64

there's one XNA value, i'll drop the row.

In [156]:
df.drop(df[df['gender']=='XNA'].index, inplace=True)
df['gender'].value_counts()

F    14236
M     7288
Name: gender, dtype: int64

checking income type values for problems

In [157]:

df['income_type'].value_counts()



employee                       11119
business                        5084
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

nothing problematic here.


there are about 70 duplicate rows. hard to know whether they are really duplicates or not because of lack of unique id, so i'll not waste time on it

In [158]:
# Checking duplicates
df.duplicated().sum()


71

Checking the size of the dataset that we now have after our first manipulations with it

In [159]:


df.shape
#previously the number of rows was 21525
print(f'{21525-df.shape[0]} rows deleted')

1 rows deleted




about 10% of the education values were lowercased.
about 5% of the children values were changed to the median value of 0
73% of days employed values were turned positive.
.4% of the dob_years values were changed to the median value
less than .005% of the gender values were changed from XNA to M (one value)
about .0035% of the data were considered duplicates and weren't dropped.







# Working with missing values

### Restoring missing values in `total_income`



total income and days employed columns have missing values. although the days employed values aren't really being used in the analysis, so will not fix them.
total income values will be filled with median or mean by certain parameter, will investigate relation to age category.




In [160]:
# Let's write a function that calculates the age category
def age_categorizer(age):
    if age < 21:
        return 'minor'
    if age < 30:
        return 'young adult'
    if age < 65:
        return 'adult'
    else:
        return 'senior'

    

In [161]:
# Test if the function works
for i in [17,25,47,90]:
    print(age_categorizer(i))


minor
young adult
adult
senior


In [162]:
# Creating new column based on function
df['age_category'] = df['dob_years'].apply(age_categorizer)
df['age_category'].value_counts()



adult          17443
young adult     3117
senior           899
minor             65
Name: age_category, dtype: int64

In [163]:
# Checking how values in the new column look
df.sample(20)



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
6951,1.0,829.887972,25,bachelor's degree,0,married,0,M,business,0,52811.813,housing transactions,young adult
6708,0.0,8619.938309,49,secondary education,1,unmarried,4,F,employee,0,72263.662,to buy a car,adult
5659,3.0,1999.778276,33,secondary education,1,civil partnership,1,F,employee,1,23242.886,to get a supplementary education,adult
10368,1.0,1564.862764,21,secondary education,1,married,0,F,employee,1,13532.439,transactions with my real estate,young adult
10175,1.0,5790.920475,34,secondary education,1,married,0,F,employee,0,22512.642,transactions with commercial real estate,adult
9276,0.0,NaN,66,secondary education,1,married,0,M,retiree,0,7565.268,real estate transactions,senior
1685,1.0,1168.253701,31,bachelor's degree,0,married,0,M,business,0,30364.068,purchase of a car,adult
8829,0.0,359.855591,56,secondary education,1,married,0,M,civil servant,0,29971.772,housing,adult
20995,0.0,2270.795656,45,secondary education,1,married,0,M,employee,0,52259.184,buy real estate,adult
17962,0.0,NaN,26,some college,2,unmarried,4,F,business,0,NaN,purchase of a car,young adult




the factors i'd consider that income depends on are:
age category,  education, number of kids, family status, income type.

In [164]:
# creating a table without missing values and making sure it's filled

df['filled_income'] = df.groupby('education')['total_income'].transform(lambda grp: grp.fillna(np.mean(grp)) )
df.loc[df['total_income'].isna()]



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category,filled_income
12,0.0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding,senior,24594.503037
26,0.0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education,adult,24594.503037
29,0.0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate,adult,24594.503037
41,0.0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase,adult,24594.503037
55,0.0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding,adult,24594.503037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2.0,NaN,47,secondary education,1,married,0,M,business,0,NaN,purchase of a car,adult,24594.503037
21495,1.0,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony,adult,24594.503037
21497,0.0,NaN,48,bachelor's degree,0,married,0,F,business,0,NaN,building a property,adult,33142.802434
21502,1.0,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate,adult,24594.503037


it looks fine

In [165]:
# Looking at the mean values for income based on our identified factors
display(df.groupby('education')['filled_income'].mean())
display(df.groupby('age_category')['filled_income'].mean())

display(df.groupby('family_status')['filled_income'].mean())
display(df.groupby('children')['filled_income'].mean())

display(df.groupby('income_type')['filled_income'].mean())


education
bachelor's degree      33142.802434
graduate degree        27960.024667
primary education      21144.882211
secondary education    24594.503037
some college           29040.132990
Name: filled_income, dtype: float64

age_category
adult          27239.294469
minor          20048.598896
senior         21918.762455
young adult    25840.071068
Name: filled_income, dtype: float64

family_status
civil partnership    26689.488496
divorced             27136.437367
married              27016.531450
unmarried            27027.581383
widow / widower      23239.634291
Name: filled_income, dtype: float64

children
0.0    26446.814369
1.0    27358.268988
2.0    27452.538207
3.0    29056.454233
4.0    27246.642148
5.0    26971.697893
Name: filled_income, dtype: float64

income_type
business                       31941.046689
civil servant                  27443.546442
employee                       25891.476391
entrepreneur                   56504.452717
paternity / maternity leave     8612.661000
retiree                        22352.577061
student                        15712.260000
unemployed                     21014.360500
Name: filled_income, dtype: float64

In [166]:
# Looking at the median values for income based on our identified factors
display(df.groupby('education')['filled_income'].median())
display(df.groupby('age_category')['filled_income'].median())

display(df.groupby('family_status')['filled_income'].median())
display(df.groupby('children')['filled_income'].median())

display(df.groupby('income_type')['filled_income'].median())

education
bachelor's degree      30338.7240
graduate degree        25161.5835
primary education      19711.1800
secondary education    23313.1460
some college           26953.8010
Name: filled_income, dtype: float64

age_category
adult          24594.503037
minor          18947.777000
senior         19505.519000
young adult    24179.295000
Name: filled_income, dtype: float64

family_status
civil partnership    24594.503037
divorced             24594.503037
married              24594.503037
unmarried            24594.503037
widow / widower      21621.862000
Name: filled_income, dtype: float64

children
0.0    24594.503037
1.0    24594.503037
2.0    24594.503037
3.0    24594.503037
4.0    24594.503037
5.0    26953.748000
Name: filled_income, dtype: float64

income_type
business                       27223.718000
civil servant                  24594.503037
employee                       24264.901000
entrepreneur                   56504.452717
paternity / maternity leave     8612.661000
retiree                        20599.628000
student                        15712.260000
unemployed                     21014.360500
Name: filled_income, dtype: float64



i've considered the income type compared to education, family status, income type, number of children and age category.
education - higher education should on average mean a higher salary.
family status - i'd think that married people would earn more, but we see that is not the case.
number of children - i'm assuming people with more children are able to maintain the family only by having a steady income.
age category - i would reason that older people had more time to build their skills, experience so it's easier for them to maintain a job with certain income.


 seeing as the mean varies the most with age category, education and income type, i'll say these are the characteristics that define income most and so i'll use the mean to fill missing values if necessary.

In [167]:
# function that we will use for filling in missing values
df.loc[df['total_income'].isna(),'total_income'] = df['filled_income']
df.loc[df['total_income'].isna()]






        
        

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category,filled_income


no na values left

In [168]:
df['total_income'].isna().value_counts()


False    21524
Name: total_income, dtype: int64

In [169]:
# Checking the number of entries in the columns
df['total_income'].count()




21524

###  Restoring values in `days_employed`

[Think about the parameters that may help you restore the missing values in this column. Eventually, you will want to find out whether you should use mean or median values for replacing missing values. You will probably conduct a research similar to the one you've done when restoring data in a previous column.]

In [170]:
df['income_type'].value_counts()
# the number of retirees is significant and their days_employed value is NaN

employee                       11119
business                        5084
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

In [171]:
# Distribution of `days_employed` medians based on our identified parameters
print(df.groupby('education')['days_employed'].median(),
      df.groupby('income_type')['days_employed'].median(),
      df.groupby('age_category')['days_employed'].median()
     )




education
bachelor's degree      1611.056758
graduate degree        3851.735057
primary education      1189.581396
secondary education    1684.271101
some college           1145.952155
Name: days_employed, dtype: float64 income_type
business                       1546.333214
civil servant                  2689.368353
employee                       1574.202821
entrepreneur                    520.848083
paternity / maternity leave    3296.759962
retiree                                NaN
student                         578.751554
unemployed                             NaN
Name: days_employed, dtype: float64 age_category
adult          1871.660146
minor           695.968951
senior         2876.221697
young adult    1012.585465
Name: days_employed, dtype: float64


In [172]:
# Distribution of `days_employed` means based on our identified parameters

print(df.groupby('education')['days_employed'].mean(),
      df.groupby('income_type')['days_employed'].mean(),
      df.groupby('age_category')['days_employed'].mean()
     )


education
bachelor's degree      2277.818710
graduate degree        3520.186537
primary education      1940.672957
secondary education    2434.270703
some college           1561.867190
Name: days_employed, dtype: float64 income_type
business                       2111.470404
civil servant                  3399.896902
employee                       2326.499216
entrepreneur                    520.848083
paternity / maternity leave    3296.759962
retiree                                NaN
student                         578.751554
unemployed                             NaN
Name: days_employed, dtype: float64 age_category
adult          2592.695308
minor           673.648361
senior         4010.478357
young adult    1220.752492
Name: days_employed, dtype: float64



since we're not gonna use days_employed in our analysis i won't bother filling in missing values for retirees even though it's a significant amount of NaN values.

In [173]:
df.loc[df['days_employed'].notna(), 'days_employed'].astype(int)

print(df.days_employed.value_counts())
df.total_income = df.total_income.astype(int, errors='ignore')

print(df.days_employed.head(10))







8437.673028    1
719.625408     1
114.463945     1
5135.928528    1
3112.788664    1
              ..
1133.987382    1
5730.178239    1
429.448441     1
1645.706859    1
1984.507589    1
Name: days_employed, Length: 15905, dtype: int64
0    8437.673028
1    4024.803754
2    5623.422610
3    4124.747207
4            NaN
5     926.185831
6    2879.202052
7     152.779569
8    6929.865299
9    2188.756445
Name: days_employed, dtype: float64


## Categorization of data




In [174]:
#1 Print the values for your selected data for categorization 1st children

df1 = df[['children','debt']]
print(df1)



       children  debt
0           1.0     0
1           1.0     0
2           0.0     0
3           3.0     0
4           0.0     0
...         ...   ...
21520       1.0     0
21521       0.0     0
21522       1.0     1
21523       3.0     1
21524       2.0     0

[21524 rows x 2 columns]


Let's check unique values

In [175]:
#1 Check the unique values children
df1['children'].value_counts()


0.0    14148
1.0     4865
2.0     2055
3.0      330
4.0       41
5.0        9
Name: children, dtype: int64


in my eyes the main groups are - childless, have 1-2 children, and child addicts (3-5 children), corresponding to expense level

now we'll categorize by groups


In [176]:
#1 Let's write a function to categorize the data based on common topics

def child_cat(n):
    if n == 0:
        return 'childless'
    if n < 4:
        return '1-3 children'
    return 'child addicts'


In [177]:
#1 Creating a column with the categories and counting the values for them

df['child_category'] = df['children'].apply(child_cat)
df1 = df[['children','debt','child_category']]
display(df1['child_category'].value_counts())




childless        14148
1-3 children      7250
child addicts      126
Name: child_category, dtype: int64

In [178]:
df1.describe()

,children,debt
count,21448.000000,21524.000000
mean,0.474357,0.080886
std,0.751427,0.272667
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,1.000000,0.000000
max,5.000000,1.000000


In [179]:
#2 Printing the values for our selected data for categorization 2nd family_status
df2 = df[['family_status','debt']]
print(df2)

           family_status  debt
0                married     0
1                married     0
2                married     0
3                married     0
4      civil partnership     0
...                  ...   ...
21520  civil partnership     0
21521            married     0
21522  civil partnership     1
21523            married     1
21524            married     0

[21524 rows x 2 columns]


In [180]:
#2 Checking the unique values family_status
df2['family_status'].value_counts()


married              12380
civil partnership     4176
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

categorizing by partnership as opposed to loners 

In [181]:
#2 Let's write a function to categorize the data based on common topics
def family_cat(fam):
    if fam in {'married', 'civil_partnership'}:
        return 'partners'
    else:
        return 'lone wolves'
family_cat('widower')

'lone wolves'

In [182]:
#2 Creating a column with the categories and counting the values for them
df['family_category'] = df['family_status'].apply(family_cat)
df2 = df[['family_status','debt','family_category']]
display(df2['family_category'].value_counts())


partners       12380
lone wolves     9144
Name: family_category, dtype: int64

In [183]:
df2.describe()

,debt
count,21524.000000
mean,0.080886
std,0.272667
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [184]:
#3 Printing the values for our selected data for categorization 3rd income_type

df3 = df[['income_type','debt']]
print(df3.head(10))


  income_type  debt
0    employee     0
1    employee     0
2    employee     0
3    employee     0
4     retiree     0
5    business     0
6    business     0
7    employee     0
8    employee     0
9    employee     0


In [185]:
#3 Checking the unique values income_type
df3['income_type'].value_counts()


employee                       11119
business                        5084
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

first group is people who aren't working full time, second is businessmen and entrepreneurs, which i consider riskier than the third group; people with regular jobs.


In [186]:
#3 Let's write a function to categorize the data based on common topics income_type
def income_cat(income_type):
    if income_type in {'retiree', 'unemployed','student','paternity / maternity leave'}:
        return 'currently not working'
    if income_type in {'entrepreneur', 'business'}:
        return 'risk takers'
    return 'regular income'
    

In [187]:
#3 Creating a column with the categories and counting the values for them, income type
df['income_category'] = df['income_type'].apply(income_cat)
df3 = df[['income_type','debt','income_category']]
display(df3['income_category'].value_counts())


regular income           12578
risk takers               5086
currently not working     3860
Name: income_category, dtype: int64

In [188]:
df3.describe()

,debt
count,21524.000000
mean,0.080886
std,0.272667
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000




categorizing the purposes:

In [189]:
df.purpose.unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

In [190]:
lemma_list_all = []

for purpose in df.purpose.unique():
    words = nltk.word_tokenize(purpose)
    lemmas = [wordnet_lemma.lemmatize(w, pos = 'n') for w in words]
    lemmas = [l.lower() for l in lemmas]
    for i in lemmas:
        if i in lemma_list_all:
            continue
        else:
            lemma_list_all.append(i)
print(lemma_list_all)

['purchase', 'of', 'the', 'house', 'car', 'supplementary', 'education', 'to', 'have', 'a', 'wedding', 'housing', 'transaction', 'having', 'for', 'my', 'family', 'buy', 'real', 'estate', 'commercial', 'residential', 'construction', 'own', 'property', 'building', 'buying', 'second-hand', 'with', 'become', 'educated', 'getting', 'an', 'ceremony', 'get', 'higher', 'profile', 'university', 'renting', 'out', 'renovation', 'going']


In [191]:
#listing purpose categories and their words

assets_category=['property','car','house','real','estate','construction','build']
education_category = ['supplementary','education','university']

Counter(lemma_list_all)

Counter({'purchase': 1,
         'of': 1,
         'the': 1,
         'house': 1,
         'car': 1,
         'supplementary': 1,
         'education': 1,
         'to': 1,
         'have': 1,
         'a': 1,
         'wedding': 1,
         'housing': 1,
         'transaction': 1,
         'having': 1,
         'for': 1,
         'my': 1,
         'family': 1,
         'buy': 1,
         'real': 1,
         'estate': 1,
         'commercial': 1,
         'residential': 1,
         'construction': 1,
         'own': 1,
         'property': 1,
         'building': 1,
         'buying': 1,
         'second-hand': 1,
         'with': 1,
         'become': 1,
         'educated': 1,
         'getting': 1,
         'an': 1,
         'ceremony': 1,
         'get': 1,
         'higher': 1,
         'profile': 1,
         'university': 1,
         'renting': 1,
         'out': 1,
         'renovation': 1,
         'going': 1})

i've decided to keep the categorization here simple, one for people buying any kind of asset, one for people looking to expand their education and one for others.
i could categorize by asset type, maybe add a category for investment (like buying real estate for renting purposes)

In [192]:
def lemmatization_func(line):
  
    words = nltk.word_tokenize(line)
    lemmas = [wordnet_lemma.lemmatize(w, pos = 'n') for w in words]
    lemmas=[l.lower() for l in lemmas]
    
    if any(word in lemmas for word in assets_category):
        return 'assets'
    elif  any(word in lemmas for word in education_category):
        return 'education'
    else:
        
        return 'other'

In [193]:
df['purpose_category'] = df.purpose.apply(lemmatization_func)
print(df.purpose_category.value_counts())

assets       13242
other         4672
education     3610
Name: purpose_category, dtype: int64


In [194]:
df4 = df[['purpose','debt','purpose_category']]
display(df4.head(10))



,purpose,debt,purpose_category
0,purchase of the house,0,assets
1,car purchase,0,assets
2,purchase of the house,0,assets
3,supplementary education,0,education
4,to have a wedding,0,other
5,purchase of the house,0,assets
6,housing transactions,0,other
7,education,0,education
8,having a wedding,0,other
9,purchase of the house for my family,0,assets


## Checking the Hypotheses


**Is there a correlation between having children and paying back on time?**

In [195]:
# Check the children data and paying back on time
display(df1.groupby('child_category')['debt'].value_counts())


# Calculating default-rate based on the number of children
display(df.groupby(['child_category'])['debt'].mean().reset_index().sort_values(by='debt'))




child_category  debt
1-3 children    0        6584
                1         666
child addicts   0         114
                1          12
childless       0       13085
                1        1063
Name: debt, dtype: int64

,child_category,debt
2,childless,0.075134
0,1-3 children,0.091862
1,child addicts,0.095238


**Conclusion**

looks like childless people have the least trouble paying their debts, which makes sense as less children means less expenses; forseen and unforseen. 
also notice slight rise in debt as number of children rise. there is a correlation between having children and paying debt on time.


**Is there a correlation between family status and paying back on time?**

In [196]:
# Checking the family status data and paying back on time
display(df2.groupby('family_status')['debt'].value_counts())


# Calculating default-rate based on family status
df.groupby(['family_category'])['debt'].mean().reset_index().sort_values(by='debt')



family_status      debt
civil partnership  0        3788
                   1         388
divorced           0        1110
                   1          85
married            0       11449
                   1         931
unmarried          0        2539
                   1         274
widow / widower    0         897
                   1          63
Name: debt, dtype: int64

,family_category,debt
1,partners,0.075202
0,lone wolves,0.088583


**Conclusion**

notice the partners have less debt. a good partnership is greater than the sum of its parts, we definitely see a correlation there.

**Is there a correlation between income level and paying back on time?**

In [197]:
# Checking the income level data and paying back on time
display(df3.groupby('income_category')['debt'].value_counts())



# Calculating default-rate based on income level

df.groupby(['income_category'])['debt'].mean().reset_index().sort_values(by='debt')



income_category        debt
currently not working  0        3642
                       1         218
regular income         0       11431
                       1        1147
risk takers            0        4710
                       1         376
Name: debt, dtype: int64

,income_category,debt
0,currently not working,0.056477
2,risk takers,0.073928
1,regular income,0.091191


**Conclusion**

looks like employees  are the real risk takers, taking on debt and defaulting on it more than the businessmen and entrepreneurs. there's some correlation although not what i'd expected. looks like non workers are more careful with their money.

**How does credit purpose affect the default rate?**

In [198]:
# Checking the percentages for default rate for each credit purpose and analyzing them
display(df4.groupby('purpose_category')['debt'].value_counts())

df.groupby(['purpose_category'])['debt'].mean().reset_index().sort_values(by='debt')



purpose_category  debt
assets            0       12186
                  1        1056
education         0        3279
                  1         331
other             0        4318
                  1         354
Name: debt, dtype: int64

,purpose_category,debt
2,other,0.075771
0,assets,0.079746
1,education,0.091690


**Conclusion**

by the default rates above we see people who are buying assets as the most unlikely to default. perhaps people who are taking a loan for educational purposes don't always end up increasing their potential for income.


# General Conclusion 


income level credit purpose

in this project we were to assess whether or not there's a connection between income level, credit purpose, number of children, family type and the default rate. 

preprocessing - 
in the education column there were too many unique values, caused by different registers which i fixed by str.lower method.

in the children column there were a few strange artifacts like 20 or -1, which may have been caused by server errors. i decided to just fill in with mean since the regular values all fall in the range of 0-5 so i treated the date as being pretty flat (no outliers)

days employed and total income had matching missing values, and lots of them which makes me believe there's something wrong with the gathering of the data or how it's being logged. will have to notify data engineers. on the same note days employed also had negative values which were fixed with boolean indexing. also changed to int instead of float values.

gender had one XNA value which was dropped (doesn't really matter since it's one value, and it really could be F or M) we also didn't use these values in our analysis.

purposes - many duplicates, similar purposes. hard work categorizing them by lemmatization. i would ask the data engineers to have a multi-choice question instead of open ended, that way we can more easily analyze, no duplicates, just categories that interest the bank.

categorization:

children - i've decided to categorize by childless, 1-2 children and 3-5 children. i tried 1-3 and 4-5 but i saw that the 4-5 children was really small, i wanted the sample to be bigger. so i moved the 3 children to the middle category.

family status - what mattered to me in the categorization is if the customer is living alone or as a strong financial unit together with another person, so i categorized by single people as opposed to partnerships.

income type - in my eyes businessmen and entrepreneurs are risk takers, employees are safe-side-leaners, and non workers have lower income.

purposes - housing and buying cars used or new got categorized as assets. i think of assets like investments, so i'd imagine this purpose category as being less risky than other expenses like weddings or renovations. and there's the education category which is also like an investment but not a physical asset.









number of children vs default rate - childless people on average defaulted less than the other categories with more children. i would say it's because of the amount of variables they handle in their lives, as in people with more children have more random events and expenses they have to deal with as opposed to a childless person. medium correlation

marital status vs default rate - difference was pretty small between the loners and the partners, in favor of the partners. in the partners we take into account people with and without children (which showed a stronger correlation). the categories are pretty broad and if we take the children amount correlation, and the fact that the categories mix childless with child-havers into consideration, it might explain why the correlation here is so weak.

income category vs default rate - the 'risk taker' category includes customers with business as the income type. which according to our grouped data default less than employed people! possibly because people who run businesses need to have financial know-how and so plan their repayment better than the employees who (in theory) worry less about financial matters, as their income is steady. the no income people includes retirees, on leave, and students. their income is probably low, so they might be taking loans more seriously and with a more concrete plan to repay, as there is less margin for error. definitely a strong correlation here.

loan purpose vs default rate - the education category loses here possibly because it's such a relatively small category, so more outliers affect the default rate for the category. most of the customers apply for loans to buy houses and cars, physical assets. perhaps it's harder to convert a mental asset (education) to profit to repay a loan. either that or it's harder to actually acquire the education (not everyone who starts studying for a degree actually get the degree in the end). would say medium low correlation

overall conclusion
if i had to put my money on one category, it'd be income type. runner-up is number of children. these would be my main parameters to look at to build a credit score. in conjunction with actual data on credit worthiness these categories would probably give an accurate view of repayment ability.

issues in the data need to be reported - negative and missing days_employed and missing total_income values.


